# Simple Classification Model

In [ ]:
from river import metrics, datasets, compose, preprocessing
from deep_river.classification import ClassifierInitialized
from torch import nn
from tqdm import tqdm

In [ ]:
dataset = datasets.Phishing()
metric = metrics.Accuracy()


class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features, 5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X


model_pipeline = compose.Pipeline(
    preprocessing.StandardScaler,
    ClassifierInitialized(
        module=MyModule(10), loss_fn="binary_cross_entropy", optimizer_fn="adam"
    ),
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  Classifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn="adam"
    lr=0.001
    output_is_logit=True
    is_class_incremental=False
    is_feature_incremental=False
    device="cpu"
    seed=42
  )
)

In [3]:
for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get()}")


0it [00:00, ?it/s]


1it [00:01,  1.06s/it]


132it [00:01, 156.37it/s]


261it [00:01, 320.51it/s]


395it [00:01, 494.88it/s]

...


920it [00:01, 1021.64it/s]


1055it [00:01, 1107.63it/s]


1185it [00:01, 1134.86it/s]


1250it [00:02, 618.42it/s] 

Accuracy: 0.7264
